In [4]:
!pip install pytube > /dev/null

In [5]:
import os
import csv
import torch
from pytube import YouTube
from google.colab import drive
from transformers import pipeline, GenerationConfig

ModuleNotFoundError: No module named 'google'

# 掛載 Google Drive

In [ ]:
driveDirectory = '/content/gdrive/'
drive.mount(driveDirectory)

In [ ]:
projectDirectory = os.path.join(driveDirectory, 'Shareddrives/ɹnoᖵoʍꓕǝǝɹɥꓕ/title_generator')
audioDirectory = os.path.join(projectDirectory, 'training_audio/')
transcriptionDirectory = os.path.join(projectDirectory, 'training_transcription/') 

# 下載 metadata

In [ ]:
!wget -q --show-progress https://raw.githubusercontent.com/lnfu/title-generator/main/training.csv training.csv
videoDataInputPath = './training.csv'

In [ ]:
!mkdir -p /content/gdrive/Shareddrives/ɹnoᖵoʍꓕǝǝɹɥꓕ/title_generator/training_transcription
!mkdir -p /content/gdrive/Shareddrives/ɹnoᖵoʍꓕǝǝɹɥꓕ/title_generator/training_audio

# 讀取 metadata

In [ ]:
with open(videoDataInputPath, 'r', encoding='utf-8') as videoDataFile:
    videoData = list(csv.reader(videoDataFile))[1:]

# 下載音檔

In [ ]:
def downloadAudio(videoId, outputDirectory):
    url = f'https://youtube.com/watch?v={videoId}'
    audio = YouTube(url).streams.get_audio_only()
    audioFilePath = audio.download(output_path=outputDirectory, filename=f'{videoId}.mp3')
    return audioFilePath

for channel, title, duration, videoId in videoData:
    audioFilePath = downloadAudio(videoId, audioDirectory)
    print(f'Audio output: {audioFilePath}')

# 語音辨識
- 使用 Whisper small (zh-tw fine-tuning)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
transcriber = pipeline(
    'automatic-speech-recognition', 
    model='Jingmiao/whisper-small-zh_tw', 
    device=device
)
transcriber.model.generation_config = GenerationConfig.from_pretrained('openai/whisper-small') 

In [ ]:
for channel, title, duration, videoId in videoData:

    audioFilePath = os.path.join(audioDirectory, f'{videoId}.mp3') 
    transcriptionFilePath = os.path.join(transcriptionDirectory, f'{videoId}.txt')
        
    transcription = transcriber(audioFilePath, return_timestamps=True)

    with open(transcriptionFilePath, 'w', encoding='utf-8') as transcriptionFile:
        transcriptionFile.write(transcription)
    
    print(f'Transcription output: {transcriptionFilePath}')